In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

In [2]:
#import to pandas
data_file = open("../../tests/inputs/out.txt")
results = pd.read_csv(data_file,header=None,sep=" ")
names = ["solution_depth","example_no","algorithm","policy","path_length","visited","processed","recursed","time"]
results.columns=names
results.head()

FileNotFoundError: [Errno 2] No such file or directory: '../../tests/inputs/out.txt'

In [ ]:
def get_color_iter(lenght):
    return iter(cm.Dark2(np.linspace(0, 1, lenght)))

In [ ]:
column_chart_name={
    "visited":"Średnia ilość odwiedzonych stanów",
    "path_length":"Średnia długość rozwiązania",
    "processed":"Średnia ilość przetworzonych stanów",
    "recursed":"Średnia maksymalna głębokość rekursji",
    "time":"Średnia czas przetwarzania [s]"
}

In [ ]:
def resultAlgFromStr(alg_name):
    return results[results.algorithm == str(alg_name)]

In [ ]:
def avg(name,data_label):
    data = resultAlgFromStr(name)
    avgs=[]
    max_solution_depth = max(results.solution_depth)
    for sol_depth in range(1,max_solution_depth+1):
        current_solution = data[data.solution_depth==sol_depth]
        avgs.append(np.average(current_solution[data_label]))
    return avgs

In [ ]:
def plotThreeAlgs(series_name):
    width = 0.25
    bfs_avg = avg("bfs",series_name)
    dfs_avg = avg("dfs",series_name)
    astr_avg = avg("astr",series_name)

    max_solution_depth = max(results.solution_depth)
    pos = np.arange(max_solution_depth)+1
    colors= get_color_iter(3)
    plt.bar(pos-(width*1.5),bfs_avg,width,label="bfs", color = next(colors),align="edge")
    plt.bar(pos,dfs_avg,width,label="dfs", color = "lime")
    plt.bar(pos+width*0.5,bfs_avg,width,label="astr", color = next(colors),align="edge")
    plt.minorticks_on()
    plt.grid(True,which="major",axis="y",ls="--", color="black")
    plt.legend(title="ALgorytm:")
    plt.xlabel("Głębokość rozwiązania")

In [ ]:
def plot_all_algs(series_name,ycale="log"):
    plotThreeAlgs(series_name)
    plt.ylabel(column_chart_name[series_name])
    plt.title("Wszystkie algorytmy: "+ column_chart_name[series_name])
    plt.yscale(ycale)
    plt.savefig("ALL_"+series_name)
    plt.show()

In [ ]:
plot_all_algs("visited")
plot_all_algs("path_length","linear")
plot_all_algs("processed")
plot_all_algs("recursed","linear")
plot_all_algs("time")

In [ ]:
overview = results.groupby(["algorithm","solution_depth"]).mean()
overview

In [ ]:
overview2 = results.groupby(["solution_depth","algorithm"],as_index=False).mean()
overview2

In [ ]:
bad_bfs = results[(results.solution_depth!=results.path_length)]
bad_bfs[bad_bfs.algorithm=="bfs"]

In [ ]:
overview3 = results.groupby(["algorithm","solution_depth","policy"],as_index=False).mean()
overview3.head()

In [ ]:
    astr = overview3[overview3.algorithm=="astr"]
    astr = astr.drop(["example_no"],axis = 1)
    hamm= astr[astr.policy=="hamm"]
    manh = astr[astr.policy=="manh"]
    manh

In [ ]:
def a_str_plot_series(series):
    width = 0.3
    colors= get_color_iter(2)
    plt.bar(manh.solution_depth-width/2,manh[series],width, label = "manhatan",color = next(colors))
    plt.bar(hamm.solution_depth+width/2,hamm[series],width, label = "hamminga", color = next(colors))
    plt.xlabel("Głębokość rozwiązania")
    plt.minorticks_on()
    plt.grid(True,which="major",axis="y",ls="--", color="black")
    plt.grid(True,which="minor",axis="y",ls="--")
    plt.legend(title="Metryki:")

In [ ]:
def astr_plot(series_name,yscale="linear"):
    a_str_plot_series(series_name)
    plt.yscale(yscale)
    plt.ylabel(column_chart_name[series_name])
    plt.title("A*:"+column_chart_name[series_name])
    plt.savefig("ASTR_"+series_name)
    plt.show()

In [ ]:
astr_plot("visited","log")
astr_plot("processed","log")
astr_plot("recursed")
astr_plot("path_length")
astr_plot("time","log")


In [ ]:
def plot_multi_bar(df, series, ylabel, yscale="log"):
    plt.rc('xtick', labelsize=15)
    plt.rc('ytick', labelsize=15)
    plt.rc('legend', fontsize=12)
    plt.rc('axes', titlesize=15)
    
    fig = plt.figure(figsize=(18,9),dpi=400)
    ax1 = fig.add_subplot(111)

    bar_x_tick = []
    bar_x_label =[]
    bar_width = 0.1
    
    policies = list(set(df.policy))
    policy_len = len(policies)
    
    bar_number = policy_len
    start_x = np.arange(1,policy_len)-(bar_number*bar_width)/2
    colors = get_color_iter(policy_len)
    
    for no,pol in enumerate(policies):
        data = df[df.policy==pol]
        bar_pos = start_x+no*bar_width
        bar_x_tick.extend(bar_pos)
        bar_x_label.extend([pol]*7)
        ax1.bar(bar_pos,
                data[series],
                bar_width,
                align="edge",
                label = pol,
                color=next(colors))

    ax1.set_xlabel("Porządki przechodzenia")
    ax1.grid(True)

    ax2 = ax1.twiny()
    ax2.set_xlabel("Głębokość rozwiązania")
    ax1.set_ylabel(ylabel)

    ax2.set_xticks(range(0,8))
    ax2.set_xbound(ax1.get_xbound())
    ax1.set_xticks(bar_x_tick)

    ax1.set_xticklabels(bar_x_label, rotation='vertical')
    ax1.set_yscale(yscale)
    ax1.legend(title="Porządki:")

In [ ]:
bfs = overview3[overview3.algorithm=="bfs"]
dfs = overview3[overview3.algorithm=="dfs"]

In [ ]:
def plot_dfs(data_series,yscale="log"): 
    plot_multi_bar(dfs,data_series,column_chart_name[data_series],yscale)
    plt.title("DFS:"+column_chart_name[data_series])
    plt.savefig("DFS_"+data_series, bbox_inches='tight')
    plt.show()

In [ ]:
def plot_bfs(data_series,yscale="log"): 
    plot_multi_bar(bfs,data_series,column_chart_name[data_series],yscale)
    plt.title("BFS:"+column_chart_name[data_series])
    plt.savefig("BFS_"+data_series, bbox_inches='tight')
    plt.show()

In [ ]:
plot_dfs("visited")
plot_dfs("path_length","linear")
plot_dfs("processed")
plot_dfs("recursed","linear")
plot_dfs("time")

In [ ]:
plot_bfs("visited")
plot_bfs("path_length","linear")
plot_bfs("processed")
plot_bfs("recursed","linear")
plot_bfs("time")